# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
#from api_keys import gmaps_key 
from api_keys import geoapify_key



In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-18.58,78,20,4.63,CA,1667702178
1,1,luanda,-8.8368,13.2343,25.00,88,75,2.06,AO,1667702525
2,2,russkaya polyana,49.4170,31.9201,2.91,83,89,3.37,UA,1667702526
3,3,putyatino,54.1600,41.1169,0.58,78,100,1.80,RU,1667702527
4,4,kapaa,22.0752,-159.3190,28.99,78,40,7.20,US,1667702425


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    #hover_cols = ["Address", "Bank Count"]
)

# Display the map plot
humidity_map


c:\Users\wolte\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\wolte\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\wolte\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\wolte\anaconda3\envs\PythonData\lib\site

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_weather_df = city_data_df[((city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 29)) 
& ((city_data_df['Humidity'] >= 30) & (city_data_df['Humidity'] <= 50)) 
& ((city_data_df['Wind Speed'] <= 10)) 
& ((city_data_df['Cloudiness'] <20))]

# Drop any rows with null values
perfect_weather_df.dropna

# Display sample data
perfect_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,busselton,-33.6500,115.3333,21.83,42,6,2.48,AU,1667702420
29,29,hobart,-42.8794,147.3294,24.47,44,0,4.63,AU,1667702196
48,48,siguiri,11.6667,-9.5000,22.34,44,8,0.62,GN,1667702549
157,157,geraldton,-28.7667,114.6000,22.23,49,0,6.69,AU,1667702343
187,187,mount gambier,-37.8333,140.7667,23.29,42,18,2.74,AU,1667702622
197,197,puerto quijarro,-17.7833,-57.7667,21.02,46,0,1.36,BO,1667702626
318,318,port lincoln,-34.7333,135.8667,23.53,46,4,3.23,AU,1667702691
411,411,doka,13.5167,35.7667,23.09,34,10,2.85,SD,1667702753
565,565,coxim,-18.5067,-54.7600,21.42,45,0,1.77,BR,1667702856


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df[["City", "Country", "Lng", "Lat", "Humidity"]].copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
19,busselton,AU,115.3333,-33.6500,42,
29,hobart,AU,147.3294,-42.8794,44,
48,siguiri,GN,-9.5000,11.6667,44,
157,geraldton,AU,114.6000,-28.7667,49,
187,mount gambier,AU,140.7667,-37.8333,42,
197,puerto quijarro,BO,-57.7667,-17.7833,46,
318,port lincoln,AU,135.8667,-34.7333,46,
411,doka,SD,35.7667,13.5167,34,
565,coxim,BR,-54.7600,-18.5067,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
busselton - nearest hotel: Broadwater Beach Resort
hobart - nearest hotel: The Old Woolstore Apartment Motel
siguiri - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
mount gambier - nearest hotel: The Old Mount Gambier Gaol
puerto quijarro - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
doka - nearest hotel: No hotel found
coxim - nearest hotel: Avendita


,City,Country,Lng,Lat,Humidity,Hotel Name
19,busselton,AU,115.3333,-33.6500,42,Broadwater Beach Resort
29,hobart,AU,147.3294,-42.8794,44,The Old Woolstore Apartment Motel
48,siguiri,GN,-9.5000,11.6667,44,No hotel found
157,geraldton,AU,114.6000,-28.7667,49,Ocean Centre Hotel
187,mount gambier,AU,140.7667,-37.8333,42,The Old Mount Gambier Gaol
197,puerto quijarro,BO,-57.7667,-17.7833,46,No hotel found
318,port lincoln,AU,135.8667,-34.7333,46,Boston Hotel
411,doka,SD,35.7667,13.5167,34,No hotel found
565,coxim,BR,-54.7600,-18.5067,45,Avendita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 10,
    color = "City",
    hover_cols = ["Humidity", "Hotel Name", "Country"]
)

# Display the map
hotel_map

c:\Users\wolte\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\wolte\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\wolte\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\wolte\anaconda3\envs\PythonData\lib\site

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)